## Nvidia Chip
## Jagadeesh Vasudevamurthy

# Write your code below
# You can use any number of private functions and classes

In [6]:
############################################################
# Exam.py 
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2025
###########################################################

############################################################
#  class Exam
###########################################################    
from collections import deque
class Exam():
    
    MIN_ID = 0
    MAX_ID = 999999999
    
    def __init__(self,show:'bool'):
        self._show = show
        #you can add whatever you want as private data or private functions
        self._root = {}      # Union-Find root
        self._rnk = {}        # Union-Find rank
        self._count = {}        # Size of each component (stored at root)
        self._nbr = {}         # Neighbor list for BFS depth calculation
        self._components = set()  # All components
        
    def _is_valid_id(self, x: int) -> bool:
        """Check if component ID is within valid range [0, 999999999]"""
        return isinstance(x, int) and self.MIN_ID <= x <= self.MAX_ID
    
    def _find(self, x):
        """Find with path compression"""
        if x not in self._root:
            return x  # Not in any group yet
        if self._root[x] != x:
            self._root[x] = self._find(self._root[x])
        return self._root[x]
    
    def _init_component(self, x):
        """Initialize a component if not seen"""
        if x not in self._root:
            self._root[x] = x
            self._rnk[x] = 0
            self._count[x] = 1
            self._nbr[x] = set()
            self._components.add(x)
        
    ############################################################
    # Group component x with component y.
    # If x and y are already in the same group (directly or indirectly), return False.
    # Otherwise, put x and y in the same group and return True.
    ############################################################
    
    def group(self, x: int, y: int) -> bool:
        
        if not self._is_valid_id(x) or not self._is_valid_id(y):
            return False
        
        # Initialize both components if not seen
        self._init_component(x)
        self._init_component(y)
        
        # Find roots FIRST (before adding edge)
        root_x = self._find(x)
        root_y = self._find(y)
        
        # Already in same group - return False WITHOUT adding edge
        if root_x == root_y:
            return False
        
        # Only add edge when actually merging (returns True)
        self._nbr[x].add(y)
        self._nbr[y].add(x)
        
        # Union by rank
        if self._rnk[root_x] < self._rnk[root_y]:
            root_x, root_y = root_y, root_x
        
        self._root[root_y] = root_x
        self._count[root_x] += self._count[root_y]
        
        if self._rnk[root_x] == self._rnk[root_y]:
            self._rnk[root_x] += 1
        
        return True
    ############################################################
    # Return the connection depth between component x and component y.
    # Return 1 if directly connected, 2 if connected through one intermediary, etc.
    # Lower values indicate more direct connectivity.
    ############################################################
    def connection_depth(self, x: int, y: int) -> int:
        
        if not self._is_valid_id(x) or not self._is_valid_id(y):
            return -1
        
        # If either not seen, return -1
        if x not in self._root or y not in self._root:
            return -1
        
        # If not in same group, return -1
        if self._find(x) != self._find(y):
            return -1
        
        # BFS for shortest path
        visited = {x}
        queue = deque([(x, 0)])
        
        while queue:
            node, depth = queue.popleft()
            if node == y:
                return depth
            for neighbor in self._nbr[node]:
                if neighbor not in visited:
                    visited.add(neighbor)
                    queue.append((neighbor, depth + 1))
        
        return -1

    ############################################################
    # Many queries may be for components that have never been connected.
    # If you do not return 1 for these, your code will fail hidden or large-scale tests.
    #
    # Return the number of elements in the group containing component a.
    # For example, if a is in a group of size 5, returns 5.
    # If a has never been seen (not connected to any other component), returns 1
    ############################################################
    def group_size(self, a: int) -> int:
        if not self._is_valid_id(a):
            return -1  # Invalid ID
        if a not in self._root:
            return 1  
        return self._count[self._find(a)]

    ############################################################
    # Return a list where each item is the size of a connected group.
    # For example, [2, 2, 1] means there are groups of sizes 2, 2, and 1.
    # Too many groups is bad for chip performance.
    ############################################################
    def component_sizes(self) -> list:
        sizes = []
        for comp in self._components:
            if self._find(comp) == comp:  # Is a root
                sizes.append(self._count[comp])
        return sizes

    ############################################################
    # return number of unique components in the chip
    ############################################################
    def n(self) -> int:
        return len(self._components)
    


##  CANNOT CHANGE ANYTHING BELOW

## TEST BENCH
## NOTHING CAN BE CHANED BELOW

In [7]:
############################################################
# ExamTest.py
# Test Bench for Exam
# Author: Jagadeesh Vasudevamurthy
# Copyright: Jagadeesh Vasudevamurthy 2025
###########################################################
 
############################################################
#  NOTHING CAN BE CHANGED IN THIS FILE
###########################################################
 
############################################################
#  All imports here
###########################################################
import sys # For getting Python Version
import random
#from Exam import *
 
 
############################################################
#  class  test factorial
###########################################################    
class Test_exam():
    def __init__(self):
        self._show = True
        self._no = 0
        self._test_simple()
        print("You got 20 marks now")
        self._test_hidden()
 
    def assert_answer(self,a:'list', b:'list'):
        sa = sorted(a)
        sb = sorted(b)
        if (sa != sb):
            print("Your answer=",a)
            print("Expected answer=",b)
            assert(0)
           
    def _test_simple(self):
        self._test1()
 
    def _test1(self)->'void':
       e = Exam(True)
       n = e.group_size(1)
       assert(n == 1)
 
       x = e.group(1,2)
       assert(x)
 
       n = e.group_size(2)
       assert(n == 2)
 
       l = e.connection_depth(1,2)
       assert(l < 5)
       x = e.group(3,4)
       assert(x)
       x = e.group(2,1)
       assert(x == False)
 
 
       l = e.connection_depth(1,2)
       assert(l < 5)
       a = e.component_sizes()
       ans = [2,2]
       self.assert_answer(a,ans)
 
       x = e.group(2,4)
       assert(x)
 
       l = e.connection_depth(1,4)
       assert(l < 5)
 
       l = e.n();
       assert(l < 5)
 
       a = e.component_sizes()
       ans = [4]
       self.assert_answer(a,ans)
 
       n = e.group_size(2)
       assert(n == 4)
 
    def _test_hidden(self):
        print("I will run hidden tests after you submit")
        print("At this point you got only 20 marks")
 
############################################################
# MAIN
###########################################################    
def main():
    t = Test_exam()
    print("EXAM ENDS. Cannot post more than once in Canvas");
    print(sys.version)
    print(
"This material is copyrighted and strictly for registered students only.\n"
"Unauthorized copying, sharing, or posting in any electronic or physical form\n"
"is a violation of USA copyright law. Violators may face fines up to $250,000\n"
"per infringement and imprisonment of up to 5 years."
)
 
 

In [8]:
############################################################
# main
###########################################################
if (__name__    == '__main__'):
    main()

You got 20 marks now
I will run hidden tests after you submit
At this point you got only 20 marks
EXAM ENDS. Cannot post more than once in Canvas
3.12.4 | packaged by Anaconda, Inc. | (main, Jun 18 2024, 10:07:17) [Clang 14.0.6 ]
This material is copyrighted and strictly for registered students only.
Unauthorized copying, sharing, or posting in any electronic or physical form
is a violation of USA copyright law. Violators may face fines up to $250,000
per infringement and imprisonment of up to 5 years.
